In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns

# Load the data
df = pd.read_csv('smoothing.csv')

g = sns.barplot(data=df, x='y_smoothing', y='mean_squared_error', palette='Set2')
plt.ylabel('Mean Squared Error', fontweight='bold')
plt.xlabel('Optimizer', fontweight='bold')
g.grid(True, which='both', linestyle='--', linewidth=1)

plt.show()